In [1]:
import sys

sys.path.append('../..')

In [2]:
from finance.discountfactor import ConstantRateDiscountFactor  

df = ConstantRateDiscountFactor(0.03)

In [6]:
import numpy as np
from scipy.stats import norm
from maths.montecarlo.processes.brownianmotions import BrownianMotion

maturity = 10.
step = 1./360
time_grid = BrownianMotion.generate_time_grid(0, maturity, step)

x_0 = [100]
mu_s = [[6]]
sigma_s = [15]

b = BrownianMotion(time_grid, x_0, mu_s, sigma_s)

In [8]:
from finance.products.european.assetswap import SwapContract

dates = [0.25*i for i in range(4)]

swap = SwapContract(b, df, dates)

In [9]:
swap.strike

100.75093045565227

In the paper of Crépey, Gerboud, Grbac and Ngor [Counterparty risk and funding:
the four wings of the TVA](http://grozny.maths.univ-evry.fr/pages_perso/crepey/papers/CVA-Wings.pdf), they compute the TVA on a swap with the following parameters (see section 4.4 Numerics):

>We choose a swap notional of $N = 310.136066$\$ so that the fixed leg of the swap is worth $100$\$ at inception

Let us compute the same way the notional such that the fixed leg is equal to $100$ \$.

\begin{align*}
P_{fixed} &=& N \times \sum_{k=1}^{N} \Delta_{T_k} \beta_{T_k} K \\
&=& (N \times K )\sum_{k=1}^{N} \Delta_{T_k} \beta_{T_k}
\end{align*}

with:
- $\Delta_{T_k} = T_{k} - T_{k-1}$
- $\beta_{t} = \exp(-\int_{0}^{t} r_s ds)$

Thus:

\begin{equation*}
N = \frac{P_{fixed}}{K \times \sum_{k=1}^{N} \Delta_{T_k} \beta_{T_k}}
\end{equation*}

In [10]:
deltas = swap.delta_time
print deltas

[ 0.25  0.25  0.25]


In [11]:
df_pillars = [df(t) for t in dates[1:]]
print df_pillars

[0.99252805481913842, 0.98511193960306265, 0.97775123719333634]


In [12]:
import numpy as np

coeff = np.dot(deltas, df_pillars)

In [13]:
N = 100. / (swap.strike * coeff)
print N

1.34337092726


In [14]:
print "P_fixed = ",N * swap.strike * coeff

P_fixed =  100.0
